In [68]:
from urllib.request import urlopen
import requests 
from bs4 import BeautifulSoup as BS
import re
import pandas as pd
import numpy as np
from collections import defaultdict
import seaborn as sns

In [69]:
req_headers = {
    'accept': '*/*',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'authority': 'www.redfin.com',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36',
    'cookie': 'RF_BROWSER_ID=L60AjcauTBebcYlIR2U4kw; RF_BID_UPDATED=1; G_ENABLED_IDPS=google; RF_VISITED=true; searchMode=1; sortOrder=1; sortOption=special_blend; collapsedMapView=1; unifiedLastSearch=name%3DSan%2520Jose%26subName%3DSan%2520Jose%252C%2520CA%252C%2520USA%26url%3D%252Fcity%252F17420%252FCA%252FSan-Jose%26id%3D2_17420%26type%3D2%26unifiedSearchType%3D2%26isSavedSearch%3D%26countryCode%3DUS; RF_MARKET=sanfrancisco; RF_BUSINESS_MARKET=2; userPreferences=parcels%3Dtrue%26schools%3Dfalse%26mapStyle%3Ds%26statistics%3Dtrue%26agcTooltip%3Dfalse%26agentReset%3Dfalse%26ldpRegister%3Dfalse%26afCard%3D2%26schoolType%3D0%26lastSeenLdp%3DnoSharedSearchCookie%26viewedSwipeableHomeCardsDate%3D1663615482894; RF_HOT_MARKET=%7B%22lastPromptedDate%22%3A1663622678439%2C%22expiration%22%3A1664160906220%2C%22promptedTimes%22%3A2%7D; RF_UNBLOCK_ID=AkKegX94; RF_CORVAIR_LAST_VERSION=437.1.0; RF_LAST_NAV=1; g_state={"i_p":1664477071453,"i_l":3}; FEED_TIMESTAMP=1663872301375; RF_LAST_SEARCHED_CITY=San%20Jose; RF_LISTING_VIEWS=157190844.156904928.156448970.157168473.157119319.153865147.156376334.157161427.157135189.157099478.156597405.156758795.157151930; RF_LAST_DP_SERVICE_REGION=482; RF_LDP_VIEWS_FOR_PROMPT=%7B%22viewsData%22%3A%7B%2209-18-2022%22%3A%7B%22156376334%22%3A2%2C%22156597405%22%3A1%2C%22156758795%22%3A1%2C%22157099478%22%3A1%2C%22157135189%22%3A1%2C%22157151930%22%3A1%2C%22157161427%22%3A3%7D%2C%2209-19-2022%22%3A%7B%22153865147%22%3A1%2C%22157119319%22%3A2%2C%22157168473%22%3A1%7D%2C%2209-20-2022%22%3A%7B%22156448970%22%3A1%2C%22156904928%22%3A1%7D%2C%2209-22-2022%22%3A%7B%22157190844%22%3A2%7D%7D%2C%22expiration%22%3A%222024-09-17T20%3A50%3A26.681Z%22%2C%22totalPromptedLdps%22%3A0%7D; AKA_A2=A; ak_bmsc=9FF122CA17FE1E89A4458C649939DF87~000000000000000000000000000000~YAAQnfEPFxH1VD+DAQAAoVfmZxHPjQWmo2t3O/XTqvZ/1SVHhr+yFLW5o0kAXot+yWYoruCrNpO3ccw5Bx/zungOUzZKcpBy1KVuHH+X484s3BoEKaMoE1BDQycWtMLopRe9kHeTaDkbosq6n2f08nxVacsMWMb+km4w8n0rWNpp5WYqxpJ4vwkgE+uAGoK2TcrprnZqO78lvCJHf97qwokJsjnMnQ59AsrzYktY7KcMvfaM2g3jh9uXhOKM5qlJeqqd+djVlMbQXzUnXt0RQ0AsIbrKpm3nPgT8Cvv4Yd0NGt7RM4M2tanq5vUjJIyvLX6XYURZGxl+QKV59gOi3mLen6e7T+LT58mkciEPt/jdXMn8c9qrDltYRZsQs99CudJdOzsfUWsiQaE=; audS=t; FEED_COUNT=%5B%229%22%2C%22f%22%5D; RF_BROWSER_CAPABILITIES=%7B%22screen-size%22%3A3%2C%22events-touch%22%3Afalse%2C%22ios-app-store%22%3Afalse%2C%22google-play-store%22%3Afalse%2C%22ios-web-view%22%3Afalse%2C%22android-web-view%22%3Afalse%7D; bm_mi=4159B310D72863C613F00E892833EFE6~YAAQnfEPF533VD+DAQAAsX3mZxFRqu7J+Yy1t52a26eVRiEDcZ1/wHmcZEsRYuhhMhPzyZBcTtSqQ+kf4ul+LqLPAVqrMD5SrZDjpwpgFb/sMJKATQqf7eE2co+Iqlt8tAZmrvLxVRttLgjMY69wkhZh3TJQBAIp5ps6k4aN7XvPWcm2VdnEA73z0Qh4DIINJ7N50EwvX0kPDWujY3EziiW313Z/P7aeXgjQfMeX5RN+lvPOnb+57eiNUPFuDpofVMAmk3Y7UO0iF762Nq2mvY1CleGGBPqmAi5XcPxO6fD3vPk6Zw2wLT/ii9nR3g==~1; bm_sv=D3347A4C765D52735869E81D6328C1D9~YAAQnfEPF573VD+DAQAAsX3mZxE41YEsMfn7RBl/6FsepnyqN8/sn0Fg9ZwhL+Ii8SxmSJvDzuHByr/PtoyiEdHTjhT3rhXN85jdDoyu7w12HoC2jUF25Sqm3JYGk7eoCs4XP0ibWuXPgf0XCzV8XYYjh3re9aITggLT3PjWu0spjSJH+MeykR3Z+2/B/THfZF6iistglIH0pgVSaQKogvcEeYphrRM6hncmdDEDBX26V5+DjewBumdroAtAACl8~1',
    'referer': 'https://www.redfin.com/',
    'sec-ch-ua': '"Google Chrome";v="105", "Not)A;Brand";v="8", "Chromium";v="105"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin'
}

In [70]:
url = "https://www.redfin.com/state/Florida/recently-sold"
r = requests.Session().get(url, headers=req_headers)
soup = BS(r.content, 'html.parser')

In [71]:
all_urls = [a['href'] for a in soup('a')  if a.has_attr('href')]
all_urls

['/',
 '/houses-near-me',
 '/rentals',
 '/feed',
 '/myredfin/favorites',
 '/premier',
 '/sell-a-home/seller-consultation/address?inquirySource=285',
 '/rentals/list-my-home-for-rent',
 'https://redfin.onelink.me/iy7h/?pid=redfin&af_channel=mobile-web-organic&c=mobile-menu',
 '/',
 '/myredfin/saved-searches',
 '/selling-options',
 '/myredfin/owner-dashboard',
 '/myredfin/openhouses',
 '/myredfin/tours',
 '/myredfin/agent',
 '/mortgage-menu',
 '/careers/real-estate',
 '/myredfin/alerts-settings',
 'tel:1-844-759-7732',
 'tel:1-844-759-7732',
 '/houses-near-me',
 '/rentals',
 '/why-sell?inquirySource=484',
 '/premier',
 '/mortgage-get-pre-approved?context=82',
 '/city/17420/CA/San-Jose/real-estate/agents',
 '/feed',
 'https://www.redfin.com/city/18142/FL/Tampa/recently-sold',
 'https://www.redfin.com/city/13655/FL/Orlando/recently-sold',
 'https://www.redfin.com/city/11458/FL/Miami/recently-sold',
 'https://www.redfin.com/city/8907/FL/Jacksonville/recently-sold',
 'https://www.redfin.com/

In [72]:
regex = r"https://www.redfin.com/city/?"
good_urls = [url for url in all_urls if re.match(regex, url)]
good_urls = list(set(good_urls))
good_urls

['https://www.redfin.com/city/6208/FL/Fort-Myers/recently-sold',
 'https://www.redfin.com/city/3344/FL/Clearwater/recently-sold',
 'https://www.redfin.com/city/8907/FL/Jacksonville/recently-sold',
 'https://www.redfin.com/city/16463/FL/Sarasota/recently-sold',
 'https://www.redfin.com/city/1903/FL/Boca-Raton/recently-sold',
 'https://www.redfin.com/city/9399/FL/Kissimmee/recently-sold',
 'https://www.redfin.com/city/6173/FL/Fort-Lauderdale/recently-sold',
 'https://www.redfin.com/city/13655/FL/Orlando/recently-sold',
 'https://www.redfin.com/city/16164/FL/St-Petersburg/recently-sold',
 'https://www.redfin.com/city/11458/FL/Miami/recently-sold',
 'https://www.redfin.com/city/19373/FL/West-Palm-Beach/recently-sold',
 'https://www.redfin.com/city/2654/FL/Cape-Coral/recently-sold',
 'https://www.redfin.com/city/4456/FL/Delray-Beach/recently-sold',
 'https://www.redfin.com/city/12171/FL/Naples/recently-sold',
 'https://www.redfin.com/city/18142/FL/Tampa/recently-sold']

In [73]:
bayarea_county_list = []
bayarea_counties = ["Fort-Myers","Clearwater","Jacksonville","Sarasota","Boca-Raton","Kissimmee","Fort-Lauderdale",
                   "Orlando","St-Petersburg","Miami","West-Palm-Beach","Cape-Coral","Delray-Beach","Naples","Tampa"]
for url in good_urls:
    if url.split("/")[6] in bayarea_counties:
        bayarea_county_list.append(url)
bayarea_county_list

['https://www.redfin.com/city/6208/FL/Fort-Myers/recently-sold',
 'https://www.redfin.com/city/3344/FL/Clearwater/recently-sold',
 'https://www.redfin.com/city/8907/FL/Jacksonville/recently-sold',
 'https://www.redfin.com/city/16463/FL/Sarasota/recently-sold',
 'https://www.redfin.com/city/1903/FL/Boca-Raton/recently-sold',
 'https://www.redfin.com/city/9399/FL/Kissimmee/recently-sold',
 'https://www.redfin.com/city/6173/FL/Fort-Lauderdale/recently-sold',
 'https://www.redfin.com/city/13655/FL/Orlando/recently-sold',
 'https://www.redfin.com/city/16164/FL/St-Petersburg/recently-sold',
 'https://www.redfin.com/city/11458/FL/Miami/recently-sold',
 'https://www.redfin.com/city/19373/FL/West-Palm-Beach/recently-sold',
 'https://www.redfin.com/city/2654/FL/Cape-Coral/recently-sold',
 'https://www.redfin.com/city/4456/FL/Delray-Beach/recently-sold',
 'https://www.redfin.com/city/12171/FL/Naples/recently-sold',
 'https://www.redfin.com/city/18142/FL/Tampa/recently-sold']

In [74]:
bayarea_county_page_links =[]
for c_l in bayarea_county_list:
    bayarea_county_page_links.append(c_l)
    for i in range(2,15):
        bayarea_county_page_links.append(c_l+'/page-'+str(i))
print(len(bayarea_county_page_links))        
print(bayarea_county_page_links)  

210
['https://www.redfin.com/city/6208/FL/Fort-Myers/recently-sold', 'https://www.redfin.com/city/6208/FL/Fort-Myers/recently-sold/page-2', 'https://www.redfin.com/city/6208/FL/Fort-Myers/recently-sold/page-3', 'https://www.redfin.com/city/6208/FL/Fort-Myers/recently-sold/page-4', 'https://www.redfin.com/city/6208/FL/Fort-Myers/recently-sold/page-5', 'https://www.redfin.com/city/6208/FL/Fort-Myers/recently-sold/page-6', 'https://www.redfin.com/city/6208/FL/Fort-Myers/recently-sold/page-7', 'https://www.redfin.com/city/6208/FL/Fort-Myers/recently-sold/page-8', 'https://www.redfin.com/city/6208/FL/Fort-Myers/recently-sold/page-9', 'https://www.redfin.com/city/6208/FL/Fort-Myers/recently-sold/page-10', 'https://www.redfin.com/city/6208/FL/Fort-Myers/recently-sold/page-11', 'https://www.redfin.com/city/6208/FL/Fort-Myers/recently-sold/page-12', 'https://www.redfin.com/city/6208/FL/Fort-Myers/recently-sold/page-13', 'https://www.redfin.com/city/6208/FL/Fort-Myers/recently-sold/page-14', 'ht

In [75]:
properties= defaultdict(set) 

for url in bayarea_county_page_links:    
    county_name = re.findall('FL/([^/]+)',url) # key       
    text = requests.get(url, headers=req_headers).text
    soup = BS(text,'html5lib')     
    county_text = soup.find('div',attrs={'id':'results-display'}) 
    all_links = [a['href'] for a in county_text('a') if a.has_attr('href')] 
    regex = r"^/FL" 
    for link in all_links:
        if re.match(regex,link):
            mlink = "https://www.redfin.com" + link
            properties[county_name[0]].add(mlink)

print(len(properties))
print(properties)

15
defaultdict(<class 'set'>, {'Fort-Myers': {'https://www.redfin.com/FL/Fort-Myers/3911-Maxine-St-33901/home/67933440', 'https://www.redfin.com/FL/Fort-Myers/9344-Independence-Way-33913/home/61866793', 'https://www.redfin.com/FL/Lehigh-Acres/8684-Athena-Ct-33971/home/61825788', 'https://www.redfin.com/FL/Fort-Myers/2436-Gorham-Ave-33907/home/61968410', 'https://www.redfin.com/FL/Fort-Myers/13252-White-Marsh-Ln-33912/unit-3204/home/67840873', 'https://www.redfin.com/FL/Fort-Myers/2236-River-Oak-Ln-33905/home/67843328', 'https://www.redfin.com/FL/Fort-Myers/9066-Prosperity-Way-33913/home/61819629', 'https://www.redfin.com/FL/Fort-Myers/9653-Hemingway-Ln-33913/unit-3309/home/61925269', 'https://www.redfin.com/FL/Fort-Myers/9625-Hemingway-Ln-33913/unit-3710/home/67644783', 'https://www.redfin.com/FL/Fort-Myers/14011-W-Hyde-Park-Dr-33912/unit-101/home/67627478', 'https://www.redfin.com/FL/Fort-Myers/6300-S-Pointe-Blvd-33919/unit-401/home/61896404', 'https://www.redfin.com/FL/Fort-Myers/133

In [78]:
from requests.exceptions import ChunkedEncodingError

# Your existing code...
properties = defaultdict(set)

for url in bayarea_county_page_links:
    county_name = re.findall('FL/([^/]+)', url)  # key
    retry_count = 3  # Number of times to retry
    while retry_count > 0:
        try:
            text = requests.get(url, headers=req_headers).text
            soup = BS(text, 'html5lib')
            county_text = soup.find('div', attrs={'id': 'results-display'})
            all_links = [a['href'] for a in county_text('a') if a.has_attr('href')]
            regex = r"^/FL"
            for link in all_links:
                if re.match(regex, link):
                    mlink = "https://www.redfin.com" + link
                    properties[county_name[0]].add(mlink)
            # Decrement retry_count to exit the loop if successful
            retry_count = 0
        except ChunkedEncodingError:
            print("Encountered ChunkedEncodingError. Retrying...")
            retry_count -= 1

# Extract the URLs from the defaultdict and store them in a list
properties_urls = []
for urls_set in properties.values():
    properties_urls.extend(urls_set)

# Print the total number of URLs and the list of URLs
print("Total URLs:", len(properties_urls))
print(properties_urls)



Total URLs: 5163
['https://www.redfin.com/FL/Fort-Myers/3911-Maxine-St-33901/home/67933440', 'https://www.redfin.com/FL/Fort-Myers/9344-Independence-Way-33913/home/61866793', 'https://www.redfin.com/FL/Lehigh-Acres/8684-Athena-Ct-33971/home/61825788', 'https://www.redfin.com/FL/Fort-Myers/2436-Gorham-Ave-33907/home/61968410', 'https://www.redfin.com/FL/Fort-Myers/13252-White-Marsh-Ln-33912/unit-3204/home/67840873', 'https://www.redfin.com/FL/Fort-Myers/2236-River-Oak-Ln-33905/home/67843328', 'https://www.redfin.com/FL/Fort-Myers/9066-Prosperity-Way-33913/home/61819629', 'https://www.redfin.com/FL/Fort-Myers/9653-Hemingway-Ln-33913/unit-3309/home/61925269', 'https://www.redfin.com/FL/Fort-Myers/9625-Hemingway-Ln-33913/unit-3710/home/67644783', 'https://www.redfin.com/FL/Fort-Myers/14011-W-Hyde-Park-Dr-33912/unit-101/home/67627478', 'https://www.redfin.com/FL/Fort-Myers/6300-S-Pointe-Blvd-33919/unit-401/home/61896404', 'https://www.redfin.com/FL/Fort-Myers/1331-Kingswood-Ct-33919/home/67

In [79]:
from splinter import Browser
from bs4 import BeautifulSoup as soup

# Initialize the list to store extracted data
data_list = []

# Set up Splinter
browser = Browser('chrome')

# Loop through each URL
for url in properties_urls:  # Use the list of generated URLs here
    # Visit the current webpage
    browser.visit(url)

    # Get the HTML content of the page
    html_content = browser.html

    # Parse the HTML content using BeautifulSoup
    page_soup = soup(html_content, 'html.parser')

    # Find the div with class "ListingStatusBannerSection" for sold date
    sold_date_div = page_soup.find('div', class_='ListingStatusBannerSection')

    # Check if sold_date_div exists before trying to access attributes
    if sold_date_div:
        # Extract the sold date
        sold_date = sold_date_div.get_text(strip=True).replace('SOLD ON', '').strip()

        # Find the first <h1> tag and its following <div> tag with a specific class
        h1_tag = page_soup.find('h1')
        if h1_tag:
            div_with_class = h1_tag.find_next('div', class_='home-main-stats-variant')  # Replace with the actual class name

            # Extract the content of <h1> and <div>
            h1_content = h1_tag.get_text()
            div_content = div_with_class.get_text() if div_with_class else ""

            # Store the data in a dictionary
            data = {'URL': url, 'sold_date': sold_date, 'h1_content': h1_content, 'div_content': div_content}
            data_list.append(data)

# Close the browser
browser.quit()

# Print the extracted data
for entry in data_list:
    print(entry['h1_content'] + ' ' + entry['div_content'] + ' ' + entry['sold_date'])


3911 Maxine St,FORT MYERS, FL 33901 $298,000Sold Price2Beds1Bath888Sq Ft SOLDON AUG 4, 2023
9344 Independence Way,FORT MYERS, FL 33913 $655,000Sold Price3Beds2Baths1,715Sq Ft SOLDON JUL 28, 2023
8684 Athena Ct,Lehigh Acres, FL 33971 $247,000Sold Price2Beds2.5Baths1,205Sq Ft SOLDON AUG 21, 2023
2436 Gorham Ave,FORT MYERS, FL 33907 $350,000Sold Price2Beds2Baths1,419Sq Ft SOLDON AUG 17, 2023
13252 White Marsh Ln #3204,FORT MYERS, FL 33912 $212,000Sold Price2Beds1Bath792Sq Ft SOLDON JUL 31, 2023
2236 River Oak Ln,Fort Myers, FL 33905 $525,000Sold Price4Beds2.5Baths3,024Sq Ft SOLDON JUL 28, 2023
9066 Prosperity Way,FORT MYERS, FL 33913 $725,000Sold Price2Beds2Baths2,052Sq Ft SOLDON JUL 27, 2023
9653 Hemingway Ln #3309,Fort Myers, FL 33913 $332,000Sold Price2Beds2Baths1,558Sq Ft SOLDON AUG 1, 2023
9625 Hemingway Ln #3710,FORT MYERS, FL 33913 $315,000Sold Price2Beds2Baths1,458Sq Ft SOLDON JUL 31, 2023
14011 W Hyde Park Dr #101,Fort Myers, FL 33912 $400,000Sold Price3Beds2.5Baths1,757Sq Ft SOL

In [80]:
# Create a DataFrame from the extracted data
df = pd.DataFrame(data_list)

# Export to Excel
excel_file_path = 'data_output.xlsx'
df.to_excel(excel_file_path, index=False)
print("Data exported to Excel:", excel_file_path)

# Export to CSV
csv_file_path = 'data_output.csv'
df.to_csv(csv_file_path, index=False)
print("Data exported to CSV:", csv_file_path)

Data exported to Excel: data_output.xlsx
Data exported to CSV: data_output.csv


In [56]:
# from splinter import Browser
# from bs4 import BeautifulSoup as soup

# # List of URLs to loop through
# urls = [
#     'https://www.redfin.com/FL/Orlando/1535-Catherine-St-32801/unit-42/home/46589653',
#     'https://www.redfin.com/FL/Orlando/2100-Winnetka-Ct-32818/home/46768694'
# ]

# # Initialize the list to store extracted data
# data_list = []

# # Set up Splinter
# browser = Browser('chrome')

# # Loop through each URL
# for url in urls:
#     # Visit the current webpage
#     browser.visit(url)

#     # Get the HTML content of the page
#     html_content = browser.html

#     # Parse the HTML content using BeautifulSoup
#     page_soup = soup(html_content, 'html.parser')

#     # Find the div with class "ListingStatusBannerSection" for sold date
#     sold_date_div = page_soup.find('div', class_='ListingStatusBannerSection')

#     # Extract the sold date
#     sold_date = sold_date_div.get_text(strip=True).replace('SOLD ON', '').strip()

#     # Find the first <h1> tag and its following <div> tag with a specific class
#     h1_tag = page_soup.find('h1')
#     if h1_tag:
#         div_with_class = h1_tag.find_next('div', class_='home-main-stats-variant')  # Replace with the actual class name

#         # Extract the content of <h1> and <div>
#         h1_content = h1_tag.get_text()
#         div_content = div_with_class.get_text() if div_with_class else ""

#         # Store the data in a dictionary
#         data = {'URL': url, 'sold_date': sold_date, 'h1_content': h1_content, 'div_content': div_content}
#         data_list.append(data)

# # Close the browser
# browser.quit()

# # Print the extracted data
# for entry in data_list:
#     #print("URL:", entry['URL'])
#     print(entry['h1_content'] + ' ' + entry['div_content'] + ' ' + entry['sold_date'])


1535 Catherine St #42,ORLANDO, FL 32801 $175,000Sold Price1Bed1Bath550Sq Ft SOLDON AUG 21, 2023
2100 Winnetka Ct,ORLANDO, FL 32818 $335,000Sold Price3Beds2Baths1,410Sq Ft SOLDON AUG 22, 2023


In [ ]:
# Creating empty lists for required parameters
State = []
County =[]
City = []
Street= []
Zipcode = []
Property_type = []
Status = []
Price = []
Year_built = []
Beds = []
Bath = []
Sqft = []
Acre=[]
Lot_area = []
HOA_due = []
School_score = []
Walk_score = []
Bike_score = []
Garage_availibility = []
Redfin_estimate = []

for county in properties:   
    
    
    for prop in properties[county]:  
        
        County.append(county)
        
        html_text = requests.get(prop,headers=req_headers).text
        soup = BS(html_text,'html5lib')
        
        try:
            state_value = soup.find('div',attrs={'class':'dp-subtext'}).text.split(', ')[1].split(' ')[0]
        except:
            state_value = np.nan
        State.append(state_value)
        
        
        try:
            city_value = soup.find('div',attrs={'class':'dp-subtext'}).text.split(', ')[0]
        except:
            city_value = np.nan
        City.append(city_value)
        
    
        try:
            street_value = soup.find('div',attrs={'class':'street-address'}).text.split(',')[0]
        except:
            street_value = np.nan
        Street.append(street_value)
        
     
        try:
            zipcode_value = soup.find('div',attrs={'class':'dp-subtext'}).text.split(', ')[-1].split(' ')[-1]
            
        except:
            zipcode_value = np.nan
        Zipcode.append(zipcode_value) 
        
        
#         try:
#             property_value = soup.findAll('div',attrs={'class':'keyDetail font-weight-roman font-size-base'})
#             for tag in property_value:
#                 if(tag.span.text == 'Property Type'):
#                     property_type_value = tag.find('span',attrs={'class':'content text-right'}).text
#         except:
#             property_type_value = np.nan
#         Property_type.append(property_type_value)       
        
        
        
#         try:
#             status_text = soup.findAll('div',attrs={'class':'keyDetail font-weight-roman font-size-base'})
#             status =''
#             for tag in status_text:
#                 if(tag.span.text=='Status'):
#                     status = tag.find('span',attrs={'class':'DefinitionFlyoutLink inline-block underline clickable'})
#                     if status == None:
#                         status = tag.find('span',attrs={'class':'content text-right'})
#         except:
#             status=np.nan
#         Status.append(status.text)     
        
        
        
        try:
            price_value = soup.find('div',attrs={'class':'statsValue'}).text.split('$')[1]
        except:
            price_value = np.nan
        Price.append(price_value)
        
        
#         try:
#             year_built_text = soup.findAll('div',attrs={'class':'keyDetail font-weight-roman font-size-base'})
#             for tag in year_built_text:
#                 if(tag.span.text=='Year Built'):
#                     year_built_value = int(tag.find('span',attrs={'class':'content text-right'}).text)
#         except:
#             year_built_value = np.nan
#         Year_built.append(year_built_value)
                
        
        try:
            bed_value_str = soup.find('div',attrs={'data-rf-test-id':'abp-beds'}).text.split('B')[0]
            bed_value = int(bed_value_str)
        except:
            bed_value=np.nan
        Beds.append(bed_value)
        
        
        try:
            bath_value_str =soup.find('div',attrs ={'data-rf-test-id':'abp-baths'}).text.split('B')[0]
            bath_value = float(bath_value_str)
        except:
            bath_value = np.nan
        Bath.append(bath_value)
        
        
        try:
            sqft_value_str =soup.find('div',attrs ={'data-rf-test-id':'abp-sqFt'}).text.split('S')[0].replace(',','')
            sqft_value = float(sqft_value_str)
        except:
            sqft_value=np.nan
        Sqft.append(sqft_value)
        
        
        try:
            acre_value =soup.find('div',attrs ={'class':'stat-block sqft-section'}).text.split('A')[0]
        except:
            acre_value=np.nan
        Acre.append(acre_value)
        
        
#         try:
#             lot_prop_text = soup.findAll('div',attrs={'class':'keyDetail font-weight-roman font-size-base'})
#             for tag in lot_prop_text:
#                 if(tag.span.text=='Lot Size'):
#                     lot_area_value = tag.find('span',attrs={'class':'content text-right'}).text
#         except:
#             lot_area_value = np.nan
#         Lot_area.append(lot_area_value)
        
        
        
        try:
            hoa_text = soup.findAll('div',attrs={'class':'keyDetail font-weight-roman font-size-base'})
            hoa_due_value =''
            for tag in hoa_text:
                if(tag.span.text=='HOA Dues'):
                    hoa_due_value = tag.find('span',attrs={'class':'content text-right'}).text
        except:
            hoa_due_value = np.nan
        HOA_due.append(hoa_due_value)
        
        
        
        try:
            school_rating = max([rating.text[0] for rating in soup.findAll('div',attrs ={'class':'gs-rating-text'})])
        except:
            school_rating = np.nan
            
        School_score.append(school_rating)        
        
                
        
        try:
            walk_score_value = int(soup.find('div',attrs={'class':'transport-icon-and-percentage walkscore'}).text.split(' /')[0])
        except:
            walk_score_value = np.nan
        Walk_score.append(walk_score_value)
        
        
        
        try:
            bike_score_value = int(soup.find('div',attrs={'class':'transport-icon-and-percentage bikescore'}).text.split(' /')[0])
        except:
            bike_score_value = np.nan
        Bike_score.append(bike_score_value)
        

        
        try:
            garage_aval = soup.findAll('span',attrs={'class':'entryItemContent'})
            for tag in garage_aval:
                if("Garage" in tag.text):
                    value = 'Yes'
                    break
                else:    
                    value = 'No'
        except:   
             value = np.nan
        Garage_availibility.append(value)
        
        
        
        try:
            redfin_estimate_text = soup.findAll('div',attrs={'class':'keyDetail font-weight-roman font-size-base'})
            redfin_estimate_value =''
            for tag in redfin_estimate_text:
                if(tag.span.text =='Redfin Estimate'):
                    redfin_estimate_value = tag.find('span',attrs={'class':'content text-right'}).text.split('$')[1]
        except:
            redfin_estimate_value=np.nan
        Redfin_estimate.append(redfin_estimate_value)
        
        

In [ ]:
df=pd.DataFrame(data={'State':State,'County':County,
                      'City':City,'Street':Street,
                      'Zipcode':Zipcode,'Property_Type':Property_type,
                      'Status':Status,
                      'Price':Price,'Year_Built':Year_built,
                      'Beds':Beds,'Bath':Bath,'Sqft':Sqft,'Acre':Acre,
                      'Lot Area':Lot_area,'HOA Due':HOA_due,'School_rating':School_score,
                      'WalkScore':Walk_score,'Bike_Score':Bike_score,
                      'Garage_availability':Garage_availibility,
                      'Redfin_estimate':Redfin_estimate
                      
})    

In [ ]:
df

In [ ]:
df.to_csv('Data_before_cleaning.csv')

In [ ]:
(df['Beds'].isna() & df['Bath'].isna()).sum()

In [ ]:
df = df.dropna(subset=['Beds', 'Bath'],how='all')

In [ ]:
df['Bike_Score'].fillna(0,inplace=True)

In [ ]:
df['Beds'].fillna(0,inplace=True)
df['Bath'].fillna(0,inplace=True)

In [ ]:
df.isna().sum()

In [ ]:
df=df.dropna(subset=['Sqft'])

In [ ]:
df['Price'] = df['Price'].astype(str).str.replace(',','')
df['Price'] = df['Price'].astype(float)

In [ ]:
#df['Redfin_estimate'] = df['Redfin_estimate'].apply(lambda x: 0 if str(x) == ',' else x)
df['Redfin_estimate'] = df['Redfin_estimate'].astype(str).str.replace(',','')
df['Redfin_estimate'] = df['Redfin_estimate'].apply(lambda x: 0 if str(x) == '' else x)
df['Redfin_estimate'].fillna(0,inplace=True)
df['Redfin_estimate'] = df['Redfin_estimate'].astype(float)

In [ ]:
df['School_rating'] = df['School_rating'].apply(lambda x: 0 if str(x) == '-' else x)
df['School_rating'].fillna(0,inplace=True)
df['School_rating'] = df['School_rating'].astype(int)

In [ ]:
df['Year_Built'].fillna(0,inplace=True)
df['Year_Built'] = df['Year_Built'].astype(int)

In [ ]:
df['HOA Due'] = df['HOA Due'].str.replace(',','')
df['HOA Due'] = df['HOA Due'].str.replace('$','')
df['HOA Due'] = df['HOA Due'].str.replace('/month','')
df['HOA Due'] = df['HOA Due'].apply(lambda x: 0 if str(x) == '' else x)
df['HOA Due'].fillna(0,inplace=True)
df['HOA Due'] = df['HOA Due'].astype(float)

In [ ]:
df['Lot Area'].fillna(0,inplace=True)
df['Lot Area'] = df['Lot Area'].str.replace(',','')
df['Lot Area'] = df['Lot Area'].str.replace('Sq. Ft.','')
df['Lot Area'] = df['Lot Area'].apply(lambda x: float(str(x)[:-6])*43560 if str(x)[-5:] == 'Acres' else (float(str(x)[:-5]) if str(x)[-4:] == 'Acre' else x))
df['Lot Area'] = df['Lot Area'].apply(lambda x: 0 if str(x) == '' else x)
df['Lot Area'] = df['Lot Area'].astype(float)

In [ ]:
df=df.drop(columns=['Acre'])

In [ ]:
df

In [ ]:
df=df.reset_index()

In [ ]:
df=df.drop(columns=['index'])

In [ ]:
df.dtypes

In [ ]:
df['Street']=df['Street'].apply(lambda x:x[0]+x[1:].lower())

In [ ]:
df['City']=df['City'].apply(lambda x:x[0]+x[1:].lower())

In [ ]:
df['State']=df['State'].apply(lambda x:x[0]+x[1:].lower())

In [ ]:
df['Property_Type']=df['Property_Type'].apply(lambda x:x[0]+x[1:].lower())

In [ ]:
df['Status']=df['Status'].apply(lambda x:x[0]+x[1:].lower())

In [ ]:
df['Garage_availability']=df['Garage_availability'].apply(lambda x:x[0]+x[1:].lower())

In [ ]:
df['County']=df['County'].apply(lambda x:x[0]+x[1:].lower())

In [ ]:
df

In [ ]:
df.duplicated().value_counts()

In [ ]:
df=df.drop_duplicates()

In [ ]:
df

In [ ]:
df.kurt(numeric_only=True)

In [ ]:
df['Price'].describe()

In [ ]:
df['Beds'].describe()

In [ ]:
df['Bath'].describe()

In [ ]:
df['Sqft'].describe()

In [ ]:
df['HOA Due'].describe()

In [ ]:
df['Lot Area'].describe()

In [ ]:
df['Redfin_estimate'].describe()

In [ ]:
sns.boxplot(x=df['Price'])

In [ ]:
sns.boxplot(x=df['Beds'])

In [ ]:
sns.boxplot(x=df['Bath'])

In [ ]:
sns.boxplot(x=df['Sqft'])

In [ ]:
sns.boxplot(x=df['HOA Due'])

In [ ]:
sns.boxplot(x=df['Lot Area'])

In [ ]:
sns.boxplot(x=df['Redfin_estimate'])

In [ ]:
Q1_Price = df['Price'].quantile(0.25)
Q3_Price = df['Price'].quantile(0.75)
IQR_Price = Q3_Price - Q1_Price
IQR_Price

In [ ]:
Q1_Beds = df['Beds'].quantile(0.25)
Q3_Beds = df['Beds'].quantile(0.75)
IQR_Beds = Q3_Beds - Q1_Beds
IQR_Beds

In [ ]:
Q1_Bath = df['Bath'].quantile(0.25)
Q3_Bath = df['Bath'].quantile(0.75)
IQR_Bath = Q3_Bath - Q1_Bath
IQR_Bath

In [ ]:
Q1_Sqft = df['Sqft'].quantile(0.25)
Q3_Sqft = df['Sqft'].quantile(0.75)
IQR_Sqft = Q3_Sqft - Q1_Sqft
IQR_Sqft

In [ ]:
Q1_HOA = df['HOA Due'].quantile(0.25)
Q3_HOA = df['HOA Due'].quantile(0.75)
IQR_HOA = Q3_HOA - Q1_HOA
IQR_HOA

In [ ]:
Q1_LotArea = df['Lot Area'].quantile(0.25)
Q3_LotArea = df['Lot Area'].quantile(0.75)
IQR_LotArea = Q3_LotArea - Q1_LotArea
IQR_LotArea

In [ ]:
Q1_RedfinEstimate = df['Redfin_estimate'].quantile(0.25)
Q3_RedfinEstimate = df['Redfin_estimate'].quantile(0.75)
IQR_RedfinEstimate = Q3_RedfinEstimate - Q1_RedfinEstimate
IQR_RedfinEstimate

In [ ]:
df['Price'] = df['Price'][~((df['Price'] < (Q1_Price - 1.5 * IQR_Price)) |(df['Price'] > 
                                                                           (Q3_Price + 1.5 * IQR_Price)))]

In [ ]:
df['Beds'] = df['Beds'][~((df['Beds'] < (Q1_Beds - 1.5 * IQR_Beds)) |(df['Beds'] > 
                                                                           (Q3_Beds + 1.5 * IQR_Beds)))]

In [ ]:
df['Bath'] = df['Bath'][~((df['Bath'] < (Q1_Bath - 1.5 * IQR_Bath)) |(df['Bath'] > 
                                                                           (Q3_Bath + 1.5 * IQR_Bath)))]

In [ ]:
df['Sqft'] = df['Sqft'][~((df['Sqft'] < (Q1_Sqft - 1.5 * IQR_Sqft)) |(df['Sqft'] > 
                                                                           (Q3_Sqft + 1.5 * IQR_Sqft)))]

In [ ]:
df['HOA Due'] = df['HOA Due'][~((df['HOA Due'] < (Q1_HOA - 1.5 * IQR_HOA)) |(df['HOA Due'] > 
                                                                           (Q3_HOA + 1.5 * IQR_HOA)))]

In [ ]:
df['Lot Area'] = df['Lot Area'][~((df['Lot Area'] < (Q1_LotArea - 1.5 * IQR_LotArea)) |(df['Lot Area'] > 
                                                                           (Q3_LotArea + 1.5 * IQR_LotArea)))]

In [ ]:
df['Redfin_estimate'] = df['Redfin_estimate'][~((df['Redfin_estimate'] < (Q1_RedfinEstimate - 1.5 * IQR_RedfinEstimate)) |(df['Redfin_estimate'] > 
                                                                           (Q3_RedfinEstimate + 1.5 * IQR_RedfinEstimate)))]

In [ ]:
df.isna().sum() #number of outliers removed

In [ ]:
df = df.dropna() #removing the null values. The outliers were removed and replaced by null values
df

In [ ]:
sns.boxplot(x=df['Price'])

In [ ]:
sns.boxplot(x=df['Beds'])

In [ ]:
sns.boxplot(x=df['Bath'])

In [ ]:
sns.boxplot(x=df['Sqft'])

In [ ]:
sns.boxplot(x=df['HOA Due'])

In [ ]:
sns.boxplot(x=df['Lot Area'])

In [ ]:
sns.boxplot(x=df['Redfin_estimate'])

In [ ]:
df.to_csv('Dataset_AfterDataCleaningFinal.csv',index=False)

In [ ]:
df